In [1]:
from dataset import IAM
from dataloader import CTCDataLoader
from model import CRNNModel

from torch.utils.data import DataLoader
from sklearn import preprocessing
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import Compose, ToTensor, Resize

In [2]:
dataset = IAM('/mnt/d/Machine-Learning/Datasets/iamdataset/uncompressed')

In [3]:
print("Size of Dataset:", len(dataset))
print("Character set:", dataset.charset)
print("Vocab size (with extra blank character for CTC):", len(dataset.charset) + 1)
dataset.data.head(10)

Size of Dataset: 13353
Character set:  !"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size (with extra blank character for CTC): 80


,Image,Segmentation,Transcription,Threshold
0,a01-000u-00.png,ok,A MOVE to stop Mr. Gaitskell from,154
1,a01-000u-01.png,ok,nominating any more Labour life Peers,156
2,a01-000u-02.png,ok,is to be made at a meeting of Labour,157
3,a01-000u-03.png,err,M Ps tomorrow. Mr. Michael Foot has,156
4,a01-000u-04.png,ok,put down a resolution on the subject,157
5,a01-000u-05.png,err,and he is to be backed by Mr. Will,156
6,a01-000u-06.png,ok,"Griffiths, M P for Manchester Exchange.",159
7,a01-000x-00.png,ok,A MOVE to stop Mr. Gaitskell from nominating,182
8,a01-000x-01.png,ok,any more Labour life Peers is to be made at a,181
9,a01-000x-02.png,ok,meeting of Labour 0M Ps tomorrow. Mr. Michael,181


In [4]:
data_loader = CTCDataLoader(dataset)
train_loader, test_loader = data_loader(split=(0.4, 0.1), batch_size=(1, 1, 1))

In [5]:
len(train_loader) + len(test_loader)

6676

In [8]:
835 * 16

13360

In [6]:
model = CRNNModel(20, 100)

In [4]:
lstm = torch.nn.LSTM(input_size=1024, hidden_size=512, num_layers=2, 
                            bidirectional=True, batch_first=True)
linear1 = torch.nn.Linear(1024, 256)
linear2 = torch.nn.Linear(256, 80)

out, _ = lstm(torch.rand(4, 32, 1024))
out = linear1(out)
out = linear2(out)
out.shape

torch.Size([4, 32, 80])